In [29]:
import yaml
import platform
import pandas as pd
import glob
from PIL import Image
import brambox as bb
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
def check_os():
    os = platform.system()

    if os == 'Darwin':
        return "MacOS"
    elif os == 'Linux':
        return "Linux"
    else:
        return "Unknown OS"
    
operating_system = check_os()


if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [31]:
from brambox.io.parser.detection import CocoParser

# Load detections

det = bb.io.load(CocoParser, '/Users/johnny/Projects/small-fast-detector/runs/detect/val/predictions.json')
print('detections:')
det['image'] = det['image'].astype(str).str.lstrip('0').astype(int)
det['class_label'] = det['class_label'].astype(int)
label_mapping = {
    0: 'person',
    1: 'car',
    2: 'truck',
    3: 'uav',
    4: 'airplane',
    5: 'boat'
}

# Applying the mapping to the 'class_label' column
det['class_label'] = det['class_label'].map(label_mapping)

display(det.head())

In [32]:
from brambox.io.parser.annotation import CocoParser
# Load annotations
anno = bb.io.load(CocoParser(add_image_dims=True), '/Users/johnny/Projects/datasets/custom_dataset_v2/annotations/instances_val2017.json')
anno['image'] = anno['image'].astype(str).str.lstrip('0').astype(int)

print('annotations:')
display(anno.head())

# save dataframes
det.to_csv('/Users/johnny/Projects/small-fast-detector/runs/detect/val/detections.csv', index=False)

anno.to_csv('/Users/johnny/Projects/small-fast-detector/runs/detect/val/annotations.csv', index=False)

In [48]:
from tqdm.notebook import tqdm


def calculate_area(row):
    return row['width'] * row['height']

def iou(box_a, box_b):
    xA = max(box_a[0], box_b[0])
    yA = max(box_a[1], box_b[1])
    xB = min(box_a[2], box_b[2])
    yB = min(box_a[3], box_b[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    boxBArea = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def calculate_map(detected, actual, class_labels):
    aps = []
    if detected.empty or 'class_label' not in detected.columns:
        return 0

    for label in class_labels:
        if label not in detected['class_label'].values:
            aps.append(0)
            continue

        dc = detected[detected.class_label == label]
        ac = actual[actual.class_label == label]

        ap_coco = []
        for iou_threshold in range(50, 100, 5):
            if dc.empty:
                ap_coco.append(0)
                continue

            pr = bb.stat.pr(dc, ac, iou_threshold / 100, smooth=True)
            ap_coco.append(bb.stat.auc_interpolated(pr))

        aps.append(sum(ap_coco) / len(ap_coco))

    mAP_coco = sum(aps) / len(aps) if aps else 0
    return mAP_coco

def calculate_pr_curve(detected, actual, iou_threshold):
    """ Calcula la curva PR para un umbral de IoU específico. """
    matched_det = bb.stat.match_det(detected, actual, threshold=iou_threshold, 
                                    criteria=bb.stat.coordinates.iou, 
                                    ignore=bb.stat.IgnoreMethod.SINGLE)
    pr_curve = bb.stat.pr(matched_det, actual)
    return pr_curve

def calculate_recall_precision(tp, fn, fp):
    """ Calcula el recall y la precisión. """
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    return recall, precision

def calculate_ap(recalls, precisions):
    """ Calcula el Average Precision (AP) a partir de las curvas de recall y precision. """
    recalls = [0] + recalls + [1] 
    precisions = [0] + precisions + [0] 

    ap = np.sum((recalls[i] - recalls[i - 1]) * precisions[i] for i in range(1, len(recalls)))
    return ap

def calculate_metrics(detected, actual, class_labels, iou_threshold=0.5):
    precision = []
    recall = []

    for label in class_labels:
        tp_per_class = 0
        fp_per_class = 0
        fn_per_class = 0
        detected_class = detected[detected['class_label'] == label]
        actual_class = actual[actual['class_label'] == label]

        for _, det_row in detected_class.iterrows():
            box_det = [det_row['x_top_left'], det_row['y_top_left'],
                       det_row['x_top_left'] + det_row['width'], 
                       det_row['y_top_left'] + det_row['height']]

            best_iou = 0
            for _, act_row in actual_class.iterrows():
                box_act = [act_row['x_top_left'], act_row['y_top_left'],
                           act_row['x_top_left'] + act_row['width'], 
                           act_row['y_top_left'] + act_row['height']]
                current_iou = iou(box_det, box_act)
                best_iou = max(best_iou, current_iou)

            if best_iou >= iou_threshold:
                tp_per_class += 1
            else:
                fp_per_class += 1

        # Calcular FN
        fn_per_class = len(actual) - tp_per_class
    
        # Calcular recall y precisión
        recall_per_class, precision_per_class = calculate_recall_precision(tp_per_class, fn_per_class, fp_per_class)
        recall.append(recall_per_class)
        precision.append(precision_per_class)
    
    recall = np.mean(recall)
    precision = np.mean(precision)

    # Calcular AP por clase y luego calcular el promedio (mAP)
    mAP = calculate_map(detected, actual, class_labels)

    return recall, precision, mAP

image_stats = pd.DataFrame(columns=['name', 'width', 'height', 'num_of_gt_objects', 'lowest_area', 'biggest_area', 'num_of_predicted_objects', 'recall', 'precision', 'mAP'])

det_grouped = det.groupby('image', observed=True)
anno_grouped = anno.groupby('image', observed=True)

class_labels = anno['class_label'].unique().tolist()

total_images = set(anno['image'].unique().tolist() + det['image'].unique().tolist())
total_images = sorted(total_images)
for image_id in tqdm(total_images):
    width = height = num_of_gt_objects = lowest_area = biggest_area = num_of_predicted_objects = np.nan
    recall = precision = mAP = 0

    if image_id in anno_grouped.groups:
        image_data = anno_grouped.get_group(image_id).copy()
        width = image_data.iloc[0]['image_width']
        height = image_data.iloc[0]['image_height']
        num_of_gt_objects = len(image_data)
        image_data['area'] = image_data.apply(calculate_area, axis=1)
        lowest_area = image_data['area'].min() if not image_data['area'].empty else np.nan
        biggest_area = image_data['area'].max() if not image_data['area'].empty else np.nan

    if image_id in det_grouped.groups:
        det_data = det_grouped.get_group(image_id)
        num_of_predicted_objects = len(det_data)
        recall, precision, mAP = calculate_metrics(det_data, image_data, class_labels)

    new_row = {
        'name': image_id,
        'width': width,
        'height': height,
        'num_of_gt_objects': num_of_gt_objects,
        'lowest_area': lowest_area,
        'biggest_area': biggest_area,
        'num_of_predicted_objects': num_of_predicted_objects,
        'recall': recall,
        'precision': precision,
        'mAP': mAP
    }
    image_stats = pd.concat([image_stats, pd.DataFrame([new_row])], ignore_index=True)


In [49]:
image_stats.sort_values(by=['name'], ascending=True, inplace=True)
image_stats

In [55]:
# max column recall
image_stats['precision'].min()

In [35]:
image_stats.to_csv('/Users/johnny/Projects/small-fast-detector/runs/detect/val/image_stats.csv', index=False)

In [36]:
full_hd_data = image_stats[(image_stats['width'] == 1920) & (image_stats['height'] == 1080)].copy()
full_hd_data = full_hd_data[full_hd_data['num_of_predicted_objects'] <= 100].copy()

In [ ]:
from ultralytics import YOLO
import os
import glob
import pandas as pd
from PIL import Image

# Suponiendo que 'root_path' está definido
images_directory = root_path + 'datasets/custom_dataset_v2/images/val/'
labels_directory = root_path + 'datasets/custom_dataset_v2/labels/val/'

image_files = glob.glob(images_directory + '*.jpg')
model = YOLO('../inference_tools/Evaluation/models/detector_best.pt', task='detect')

df_rows = []

def get_image_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

for image_file in image_files:
    image_resolution = get_image_resolution(image_file)
    base_name = os.path.basename(image_file).replace('.jpg', '')
    label_file = os.path.join(labels_directory, base_name + '.txt')
    
    if os.path.exists(label_file):
        with open(label_file, 'r') as file:
            annotation_data = file.readlines()
        
        for line in annotation_data:
            class_id, x_center, y_center, width, height = line.strip().split()
            object_width = int(float(width) * image_resolution[0])
            object_height = int(float(height) * image_resolution[1])

            df_rows.append({
                'file_name': base_name + '.jpg',
                'class_id': int(class_id),
                'x_center': float(x_center),
                'y_center': float(y_center),
                'width': float(width),
                'height': float(height),
                'res_width': image_resolution[0],
                'res_height': image_resolution[1],
                'obj_width': object_width,
                'obj_height': object_height,
                'image_path': image_file,
                'label_path': label_file,
            })

df_annotations = pd.DataFrame(df_rows)

def calculate_area(width, height):
    return width * height

def predict_yolov8(image_path, label_path=None):
    results = model(image_path, size=640)
    
    predictions = []  
    metrics = {'recall': 0.0, 'map': 0.0}  
    return predictions, metrics

# Procesamiento adicional para obtener las métricas y predicciones
for index, row in df_annotations.iterrows():
    predictions, metrics = predict_yolov8(row.image_path)

    df_annotations.at[index, 'num_of_predicted_objects'] = len(predictions)
    df_annotations.at[index, 'recall'] = metrics['recall']
    df_annotations.at[index, 'map'] = metrics['map']

    areas = [calculate_area(obj.obj_width, obj.obj_height) for obj in df_annotations.itertuples() if obj.image_path == row.image_path]
    if areas:
        df_annotations.at[index, 'lowest_area'] = min(areas)
        df_annotations.at[index, 'biggest_area'] = max(areas)

df_annotations['num_of_gt_objects'] = df_annotations.groupby('file_name')['file_name'].transform('count')

print(df_annotations.head())

In [ ]:
df_annotations.to_csv('/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/annotations_valid.csv', index=False)

In [ ]:
full_hd_new_data = df_annotations[(df_annotations['res_width'] == 1920) & (df_annotations['res_height'] == 1080)].copy()